# Acquisition des additifs et de leurs toxicités
https://www.quechoisir.org/comparatif-additifs-alimentaires-n56877/
Il manque 333 additifs dans la liste que choisir
combiné avec :
https://www.les-additifs-alimentaires.com/
+ recherches diverses 
Total : 691 additifs et leur toxicité répertorié dans le CSV

In [1]:
%run ./Desu-Step-0-Init.ipynb

In [2]:
df_additifs = pd.read_csv (filepath_data+"Additif_toxicite.csv", 
                           encoding='ISO-8859-1', 
                           delimiter =";",
                           decimal=",",
                           dtype = {'Score': float})
df_additifs["Code"]=df_additifs["Code"].str.upper()

### Creation d'un DF d'interim pour intégration du scoring en DB
437285 Lignes ont des additifs

In [3]:
debug ("Debut chargement dataframe")
df_food_add = pd.read_sql_query    ('select '+
                                    'code,'+
                                    'product_name,'+
                                    'additives_tags,'+
                                    'score_additifs,'+ 
                                    'nbr_additifs,'+ 
                                    'tox_moy_additifs '+
                                    'from desu.food where additives_tags !="nan"',db) 
# Impossible de faire la distinction entre les produits sans additifs et les produits dont les additifs n'ont pas été renseignés 
# dans la base de données.

debug("Taille du dataframe après chargement:"+str(round(df_food_add.memory_usage().sum() / (1024**2),1))+"Mo") #converting to megabyte
debug ("shape du dataframe :"+str(df_food_add.shape))
df_food_add.head()

2022-06-05 18:17:09 Debug -> Debut chargement dataframe
2022-06-05 18:17:42 Debug -> Taille du dataframe après chargement:20.0Mo
2022-06-05 18:17:42 Debug -> shape du dataframe :(437285, 6)


,code,product_name,additives_tags,score_additifs,nbr_additifs,tox_moy_additifs
0,00000000001,hyde icon,"en:e296,en:e330,en:e552,en:e950,en:e955",,,
1,0000000001199,Solène céréales poulet,"en:e150,en:e160a,en:e202,en:e316,en:e466",,,
2,0000000001663,Crème dessert chocolat,en:e406,,,
3,0000000003827,Suedois saumon,"en:e300,en:e503",,,
4,0000000004510,Salade shaker taboulé,en:e202,,,
5,0000000004559,Peanuts,"en:e500,en:e500ii,en:e503,en:e503ii",,,
6,0000000005272,Sandwich solene céréales sicilien,"en:e250,en:e252",,,
7,0000000005470,BAguette bressan,"en:e14xx,en:e262,en:e262i,en:e326",,,
8,0000000016124,Organic Muesli,en:e306,,,
9,0000000018289,Antioxidant Mix - Berries & Chocolate,"en:e322,en:e322i",,,


# Enrichissement 1 : Scoring des références sur la base des additifs et ajout en DB

In [ ]:
missing_adt = []
my_update_base = 'update desu.food set score_additifs={sa},nbr_additifs={na},tox_moy_additifs={ta} where code = "{cd}" ;'
for row in range (len(df_food_add)) :
    score = 0
    if 
    for addit in df_food_add.loc[row,"additives_tags"].split(",") :
        try : # certains additifs sont absent de la liste QUE CHOISIR
            valeur_additifs = df_additifs.loc[df_additifs["Code"]==addit,"Score"].values[0].item()
            score +=valeur_additifs
            # debug(str(df_food_add.loc[row,"product_name"])+" / "+str(addit)+" / "+str(valeur_additifs))
        except :
            if addit not in missing_adt :
                missing_adt.append(addit)
    sa = score
    na = len(df_food_add.loc[row,"additives_tags"].split(","))
    ta = score/len(df_food_add.loc[row,"additives_tags"].split(","))
    
    # Mise à jour du dataframe
    df_food_add.at[row,"score_additifs"]  =sa
    df_food_add.at[row,"nbr_additifs"]    =na
    df_food_add.at[row,"tox_moy_additifs"]=ta
    
    # Mise à jour de la db
    my_update = my_update_base.format (sa=sa,
                                       na=na,
                                       ta=ta,
                                       cd=df_food_add.loc[row,"code"])
    exec_sql (my_update,2)

    print (str(row)+"/"+str(len(df_food_add)),end='\r')

if len(missing_adt)>0 :
    debug ("liste des additifs inconnus :"+str(missing_adt))

In [4]:
df_food_add.head()

,code,product_name,additives_tags,score_additifs,nbr_additifs,tox_moy_additifs
0,00000000001,hyde icon,"en:e296,en:e330,en:e552,en:e950,en:e955",,,
1,0000000001199,Solène céréales poulet,"en:e150,en:e160a,en:e202,en:e316,en:e466",,,
2,0000000001663,Crème dessert chocolat,en:e406,,,
3,0000000003827,Suedois saumon,"en:e300,en:e503",,,
4,0000000004510,Salade shaker taboulé,en:e202,,,


# Ajout du nombre d'ingrédient

In [ ]:
debug ("Debut chargement dataframe")
df_food_ing = pd.read_sql_query    ('select '+
                                    'code,'+
                                    'product_name,'+
                                    'ingredients_tags,'+
                                    'nbr_ingredients '+
                                    'from desu.food where ingredients_tags !="nan"',db) 
# Impossible de faire la distinction entre les produits sans additifs et les produits dont les additifs n'ont pas été renseignés 
# dans la base de données.

debug("Taille du dataframe après chargement:"+str(round(df_food_ing.memory_usage().sum() / (1024**2),1))+"Mo") #converting to megabyte
debug ("shape du dataframe :"+str(df_food_ing.shape))
# compter 5 min pour le chargement
df_food_ing.head(10)

In [ ]:
my_update_base = 'update desu.food set nbr_ingredients={ni} where code = "{cd}" ;'

In [ ]:
for row in range (len(df_food_ing)) :
    ni = len(df_food_ing.loc[row,"ingredients_tags"].split(","))
    df_food_ing.at[row,"nbr_ingredients"]  = ni

    exec_sql (my_update_base.format (ni=ni,cd=df_food_ing.loc[row,"code"]),2)
    
    print (str(row)+"/"+str(len(df_food_ing)),end='\r')


In [ ]:
df_food_ing.head(10)